In [7]:
import pandas as pd
import os
import datetime as dt

In [452]:
#from target exploration!!!!

def mod_concat_NEM(file_path):
    df = pd.read_pickle(file_path)
    df.drop(columns=["BandNo","RRNBandPrice","BandCost"], inplace=True)
    df = df.set_index('PeriodID', drop=True)
    df.index = pd.to_datetime(df.index).tz_localize(None)
    df[["Price", "Increase"]] = df[["Price", "Increase"]].astype(float)
    df.sort_index(inplace=True, ascending=False)
    df.index = df.index - dt.timedelta(minutes = 5)
    df = df.loc[df.index < dt.datetime(2018,8,1), :]
    assert df["Price"].dtype == float
    assert df["Increase"].dtype == float
    assert df.isna().any().any() == False
    return df

In [453]:
file = '{}/nem-data/nemde_clean/concat_for_energy_SA1'.format(os.environ['HOME'])
df = mod_concat_NEM(file)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5951880 entries, 2015-06-23 13:40:00 to 2012-01-11 12:45:00
Data columns (total 6 columns):
RegionID            object
Market              object
Price               float64
Unit                object
DispatchedMarket    object
Increase            float64
dtypes: float64(2), object(4)
memory usage: 317.9+ MB
None


In [463]:
df.reset_index(inplace=True)

In [464]:
df.rename(columns={"Unit":"DUID"}, inplace=True)

In [465]:
df.head()

,PeriodID,RegionID,Market,Price,DUID,DispatchedMarket,Increase
0,2018-07-31 23:55:00,SA1,Energy,70.70204,ER01,ENOF,0.34712
1,2018-07-31 23:55:00,SA1,Energy,70.70204,ER04,ENOF,0.31058
2,2018-07-31 23:55:00,SA1,Energy,70.70204,ER03,ENOF,0.34712
3,2018-07-31 23:55:00,SA1,Energy,70.70204,ER02,ENOF,0.31058
4,2018-07-31 23:50:00,SA1,Energy,69.24109,DEVILS_G,L5MI,-1.29132


In [466]:
df_DUID_CO2_overtime = pd.merge(df, CO2_generators_grouped, how="inner", on="DUID")

In [467]:
df_DUID_CO2_overtime.head()

,PeriodID,RegionID,Market,Price,DUID,DispatchedMarket,Increase,CO2E_EMISSIONS_FACTOR
0,2018-07-31 23:55:00,SA1,Energy,70.70204,ER01,ENOF,0.34712,0.910143
1,2018-07-31 23:50:00,SA1,Energy,69.24109,ER01,ENOF,0.64566,0.910143
2,2018-07-31 23:50:00,SA1,Energy,69.24109,ER01,L5RE,0.64566,0.910143
3,2018-07-31 23:45:00,SA1,Energy,69.94850,ER01,ENOF,0.34342,0.910143
4,2018-07-31 23:15:00,SA1,Energy,47.00597,ER01,ENOF,0.23545,0.910143


In [440]:
count = 0
lst1 = []
for i in set(df.Unit):
    if i in set(CO2_generators_grouped.DUID):
        count += 1
        lst1.append(i)
count

177

In [421]:
count = 0
lst2 = []
for i in set(df.Unit):
    if i in set(CO2_generators.GENSETID):
        count += 1
        lst2.append(i)
count

163

In [422]:
count = 0
lst_diff =[]
for i in lst2:
    if i not in lst1:
        count += 1
        lst_diff.append(i)
lst_diff

['ANGAS1', 'MCKAY2', 'ANGAS2']

In [438]:
CO2_gensetid_group = CO2_generators.groupby("GENSETID").mean().reset_index()
CO2_gensetid_group

,GENSETID,CO2E_EMISSIONS_FACTOR
0,AGLHAL,1.174440
1,AGLNOW1,0.062000
2,AGLSITA1,0.062000
3,AGLSOM,0.817915
4,ANGAS1,1.013600
...,...,...
423,YWNL1,1.315515
424,YWPS1,1.315515
425,YWPS2,1.315515
426,YWPS3,1.315515


In [441]:
CO2_generators_grouped
for i,j in CO2_gensetid_group.GENSETID.iteritems():
    if j in lst_diff:
        CO2_generators_grouped = pd.concat([g, CO2_generators.iloc[[i],:].loc[:,["GENSETID","CO2E_EMISSIONS_FACTOR"]].rename(columns={"GENSETID":"DUID"})],)
CO2_generators_grouped = CO2_generators_grouped.sort_values(by="DUID").reset_index(drop=True)
CO2_generators_grouped

,DUID,CO2E_EMISSIONS_FACTOR
0,AGLHAL,1.174440
1,AGLNOW1,0.062000
2,AGLSITA1,0.062000
3,AGLSOM,0.817915
4,ANGAS1,1.013600
...,...,...
383,YWNL1,1.315515
384,YWPS1,1.315515
385,YWPS2,1.315515
386,YWPS3,1.315515


In [445]:
CO2_generators_grouped[CO2_generators_grouped.DUID == "DEVILS_G"]

,DUID,CO2E_EMISSIONS_FACTOR
89,DEVILS_G,0.0


In [13]:
path = '{}/Desktop/Portfolio_project/DUID_CO2.csv'.format(os.environ['HOME'])

In [418]:
CO2_generators = pd.read_csv(path)
CO2_generators.head()

(428, 6)


,DUID,GENSETID,REGIONID,CO2E_EMISSIONS_FACTOR,CO2E_ENERGY_SOURCE,CO2E_DATA_SOURCE
0,AGLNOW1,AGLNOW1,NSW1,0.062,Landfill biogas methane,NGA 2018
1,AGLSITA1,AGLSITA1,NSW1,0.062,Landfill biogas methane,NGA 2018
2,APPIN,APPIN,NSW1,0.765,Coal seam methane,NGA 2018
3,BDONGHYD,BDONGHYD,NSW1,0.000,Hydro,ISP 2018
4,BERYLSF1,BERYLSF1,NSW1,0.000,Solar,ISP 2018


In [439]:
CO2_generators_grouped = CO2_generators.groupby("DUID").mean()
CO2_generators_grouped = CO2_generators_grouped.sort_values(by="DUID").reset_index()
CO2_generators_grouped.head()

,DUID,CO2E_EMISSIONS_FACTOR
0,AGLHAL,1.174440
1,AGLNOW1,0.062000
2,AGLSITA1,0.062000
3,AGLSOM,0.817915
4,ANGAST1,1.013600


In [129]:
CO2_generators_grouped.shape

(384, 1)

In [209]:
CO2_generators_unique = CO2_generators.drop_duplicates("DUID")
CO2_generators_unique = CO2_generators_unique.sort_values(by="DUID").reset_index(drop=True)
CO2_generators_unique.head()

,DUID,GENSETID,REGIONID,CO2E_EMISSIONS_FACTOR,CO2E_ENERGY_SOURCE,CO2E_DATA_SOURCE
0,AGLHAL,AGLHAL,SA1,1.174440,Natural Gas (Pipeline),ISP 2018
1,AGLNOW1,AGLNOW1,NSW1,0.062000,Landfill biogas methane,NGA 2018
2,AGLSITA1,AGLSITA1,NSW1,0.062000,Landfill biogas methane,NGA 2018
3,AGLSOM,AGLSOM,VIC1,0.817915,Natural Gas (Pipeline),ISP 2018
4,ANGAST1,ANGAS1,SA1,1.013600,Diesel oil,ISP 2018


In [122]:
CO2_generators.DUID.nunique() #assert

384

In [254]:
count = 0
lst_i = []
for i in CO2_generators_unique.CO2E_EMISSIONS_FACTOR.index:
    if abs(CO2_generators_unique.CO2E_EMISSIONS_FACTOR[i] - CO2_generators_grouped.CO2E_EMISSIONS_FACTOR[i]) > 0.001:
        lst_i.append(i)
        r = CO2_generators_unique.iloc[[i]]
r

,DUID,GENSETID,REGIONID,CO2E_EMISSIONS_FACTOR,CO2E_ENERGY_SOURCE,CO2E_DATA_SOURCE
17,BBASEHOS,BBASEH_1,VIC1,0.601,Natural Gas (Pipeline),NGA 2018


In [247]:
CO2_generators_grouped.iloc[[17],:]

,DUID,CO2E_EMISSIONS_FACTOR
17,BBASEHOS,0.719667


In [252]:
CO2_generators_grouped.iloc[[0,4,7,8]]

,DUID,CO2E_EMISSIONS_FACTOR
0,AGLHAL,1.17444
4,ANGAST1,1.01360
7,BAKING1,0.00000
8,BALBG1,0.00000


In [214]:
CO2_generators_pricediff = CO2_generators_grouped[(CO2_generators_unique.CO2E_EMISSIONS_FACTOR - CO2_generators_grouped.CO2E_EMISSIONS_FACTOR) > 0.001]
CO2_generators_pricediff.head()

,DUID,CO2E_EMISSIONS_FACTOR


In [201]:
CO2_generators_pricediff1 = CO2_generators_grouped[abs(CO2_generators_unique.CO2E_EMISSIONS_FACTOR - CO2_generators_grouped.CO2E_EMISSIONS_FACTOR) > 0.01]
CO2_generators_pricediff1.head()

,DUID,CO2E_EMISSIONS_FACTOR
0,AGLHAL,1.174440
2,AGLSITA1,0.062000
3,AGLSOM,0.817915
4,ANGAST1,1.013600
5,APPIN,0.765000


In [96]:
DUID_counts = CO2_generators.DUID.value_counts()
DUID_counts.index[DUID_counts[:] != 1]
print(DUID_counts)

MCKAY1      5
SITHE01     4
DDPS1       4
SHGEN       3
GANNSF1     3
           ..
BROWNMT     1
HALAMRD1    1
MACKNTSH    1
JACKSGUL    1
CALLNL1     1
Name: DUID, Length: 384, dtype: int64


In [160]:
CO2_generators.DUID == "DDPS"

0      False
1      False
2      False
3      False
4      False
       ...  
423    False
424    False
425    False
426    False
427    False
Name: DUID, Length: 428, dtype: bool

In [260]:
for i in CO2_generators.DUID:
    #print(type(i))

SyntaxError: unexpected EOF while parsing (<ipython-input-260-fa25c311bd1b>, line 2)

In [294]:
DUID_counts.index

Index(['MCKAY1', 'SITHE01', 'DDPS1', 'SHGEN', 'GANNSF1', 'CPSA', 'PPCCGT',
       'SNUG1', 'BBASEHOS', 'SMCSF1',
       ...
       'YWPS2', 'MURAYNL1', 'HDWF3', 'VPGS5', 'TREVALLN', 'BROWNMT',
       'HALAMRD1', 'MACKNTSH', 'JACKSGUL', 'CALLNL1'],
      dtype='object', length=384)

In [307]:
CO2_generators[CO2_generators.DUID == "SNUG1"]

,DUID,GENSETID,REGIONID,CO2E_EMISSIONS_FACTOR,CO2E_ENERGY_SOURCE,CO2E_DATA_SOURCE
270,SNUG1,SNUG1,SA1,1.448351,Diesel oil,ISP 2018
271,SNUG1,SNUG2,SA1,1.448351,Diesel oil,ISP 2018
272,SNUG1,SNUG3,SA1,1.448351,Diesel oil,ISP 2018


In [315]:
lst = ["DDPS1", "SNUG1"]
#lst.index("SNUG1")
CO2_generators[[CO2_generators.DUID == lst.index(i) for i in lst]]

ValueError: Item wrong length 2 instead of 428.

In [292]:
len(CO2_generators.DUID[DUID_counts.index])

384

In [293]:
len(CO2_generators)

428

In [11]:
DUID_CO2_identification = CO2_generators[["DUID","CO2E_EMISSIONS_FACTOR"]]
DUID_CO2_identification

,DUID,CO2E_EMISSIONS_FACTOR
0,AGLNOW1,0.062000
1,AGLSITA1,0.062000
2,APPIN,0.765000
3,BDONGHYD,0.000000
4,BERYLSF1,0.000000
...,...,...
423,YWNL1,1.315515
424,YWPS1,1.315515
425,YWPS2,1.315515
426,YWPS3,1.315515


In [18]:
path = '{}/Desktop/Portfolio_project/Fuel_sources.csv'.format(os.environ['HOME'])

In [19]:
fuel_source = pd.read_csv(path)
fuel_source

,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Physical Unit No.,Unit Size (MW),Aggregation,DUID,Reg Cap (MW),Max Cap (MW),Max ROC/Min
0,AETV Pty Ltd,Bairnsdale Power Station,VIC1,Generator,Market,Scheduled,Fossil,Natural Gas,Combustion,Open Cycle Gas turbines (OCGT),1,47,Y,BDL01,47,47,4
1,AETV Pty Ltd,Bairnsdale Power Station,VIC1,Generator,Market,Scheduled,Fossil,Natural Gas,Combustion,Open Cycle Gas turbines (OCGT),2,47,Y,BDL02,47,47,4
2,AETV Pty Ltd,Bell Bay Three Power Station,TAS1,Generator,Market,Scheduled,Fossil,Natural Gas,Combustion,Open Cycle Gas turbines (OCGT),1,35,Y,BBTHREE1,35,49,10
3,AETV Pty Ltd,Bell Bay Three Power Station,TAS1,Generator,Market,Scheduled,Fossil,Natural Gas,Combustion,Open Cycle Gas turbines (OCGT),2,35,Y,BBTHREE2,35,49,10
4,AETV Pty Ltd,Bell Bay Three Power Station,TAS1,Generator,Market,Scheduled,Fossil,Natural Gas,Combustion,Open Cycle Gas turbines (OCGT),3,35,Y,BBTHREE3,35,49,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,Willogoleche Power Pty Ltd as Trustee for the ...,Willogoleche Wind Farm,SA1,Generator,Market,Semi-Scheduled,Wind,Wind,Renewable,Wind - Onshore,25-32,3.43,Y,WGWF1,27.44,119,50
474,Wilmar Sugar Pty Ltd,Victoria Mill,QLD1,Generator,Non-Market,Non-Scheduled,Renewable/ Biomass / Waste,Bagasse,Combustion,Steam Super Critical,3,5,N,-,24,24,0
475,Wilmar Sugar Pty Ltd,Victoria Mill,QLD1,Generator,Non-Market,Non-Scheduled,Renewable/ Biomass / Waste,Bagasse,Combustion,Steam Super Critical,4,19,N,-,-,-,-
476,Windy Hill Wind Farm Pty Ltd,Windy Hill Wind Farm,QLD1,Generator,Market,Non-Scheduled,Wind,Wind,Renewable,Wind - Onshore,1-20,0.6,Y,WHILL1,12,12,NaN
